# Scissor Jack Optimizer

In [2]:
# import packages here
using SNOW
using Plots

In [3]:
# Definition of structs for components of jack
"""
Diagonal member
This is the struct object for the diagonal members. It assumes that they are all U-bars

t is the thickness of the diagonal member in inches
w is the width of the diagonal member in inches
S_y is the yield strength of the diagonal member in psi
ppi is the price per inch of the diagonal member in USD/inch
"""
struct DiagonalMember
    t::Float32
    w::Float32
    S_y::Float32
    ppi::Float32
end

"""
Crossbar
This is the type for the crossbars.
"""
abstract type Crossbar end

"""
CrossbarSquare
This is the struct for the hollow square crossbars. It assumes that they are all square bars.

t is the thickness of the crossbar in inches
w is the width of the crossbar in inches
S_y is the yield strength of the crossbar in psi
E is the modulus of elasticity of the crossbar in psi
ppi is the price per inch of the crossbar in USD/inch
"""
struct CrossbarSquare <: Crossbar
    t::Float32
    w::Float32
    S_y::Float32
    E::Float32
    ppi::Float32
end

"""
Crossbar Round
This is the struct for the solid round crossbars. It assumes that they are all solid round bars.

d is the diameter of the crossbar in inches
S_y is the yield strength of the crossbar in psi
E is the modulus of elasticity of the crossbar in psi
ppi is the price per inch of the crossbar in USD/inch
"""
struct CrossbarRound <: Crossbar
    d::Float32
    S_y::Float32
    E::Float32
    ppi::Float32
end

"""
Pin
This is the struct object for the pins.

d is the diameter of the pin in inches
S_y is the yield strength of the pin in psi
p is the price for a singular pin in USD
"""
struct Pin
    t::Float32
    w::Float32
    S_y::Float32
    E::Float32
    ppi::Float32
end

# definition of Best Solution So Far (BSSF) struct
"""
Solution
This is the struct object for the Best Solution So Far (BSSF)

p is the price
diag is the struct object for the diagonal member
cross is the struct object for the crossbar
pin is the struct object for the pin
len_d is the length of the diagonal members
len_cb is the length of the crossbar
hole_d is the length from the center of the hole to the edge of the member for the diagonal members
hole_cb is the length from the center of the hole to the edge of the member for the crossbar
"""
struct Solution #updated such that the default Crossbar is round
    p::Float16
    diag::DiagonalMember
    cross::CrossbarRound
    pin::Pin
    len_d::Float32
    len_cb::Float32
    hole_d::Float32
    hole_cb::Float32
end

#definition of struct for the safety profile
"""
SafetyProfile
this is the struct object that holds the factors of safety for the failure modes

diag_tearout is the factor of safety for the diagonal member in tearout
diag_axial is the factor of safety for the axial forces in the diagonal member
diag_bearing is the factor of safety for bearing in the diagonal member
cross_bearing is the factor of safety for bearing in the crossbar
cross_tearout is the factor of safety for tearout in the crossbar
cross_buckling is the factor of safety for buckling in the crossbar
pin_shear is the factor of safety for direct shear in the pin
pin_bearing is the factor of safety for bearing in the pin
"""
struct SafetyProfile
    diag_tearout::Float32
    diag_axial::Float32
    diag_bearing::Float32
    cross_bearing::Float32
    cross_tearout::Float32
    cross_buckling::Float32
    pin_shear::Float32
    pin_bearing::Float32
end

SafetyProfile

In [4]:
#############################################################################
# Stress functions:
#############################################################################
"""
scissor_forces(applied_force, cross_bar_length; diagonal_length = 8.5)
This function returns the force in the diagonal and cross-bar members of a scissor jack.

applied_force is the tension force applied to the jack in Newtons
cross_bar_length is the length of the cross bar in inches
diagonal_length is the length of the diagonal bar in inches. Its default length is 8.5 inches.
"""
function scissor_forces(applied_force, cross_bar_length; diagonal_length = 8.5)
    alpha = acos(cross_bar_length / (2 * diagonal_length)) #rad
    F_d = (1/2) * (applied_force / sin(alpha)) #lbf
    F_cb = applied_force * cot(alpha) #lbf
    return F_d, F_cb
end

"""
diagonal_bar_bearing_stress(force, diameter, thickness)
This function returns the bearing stress in the diagonal bar

force is the force in the diagonal member
diameter is the diameter of the hole in the diagonal member
thickness is the thickness of the diagonal member
"""
function diagonal_bar_bearing_stress(force, diameter, thickness)
    return (force) / (2 * diameter * thickness)
end

"""
cross_bar_bearing_stress(force, diameter, thickness), FOR SQUARE CROSSBAR
This function returns the bearing stress in the crossbar for a square shape

force is the force in the crossbar
diameter is the diameter of the hole in the crossbar
thickness is the thickness of the crossbar
"""
function cross_bar_bearing_stress(force, diameter, thickness)
    return (force) / (2 * diameter * thickness)
end

"""
cross_bar_bearing_stress_round(force, diameter, diameter_crossbar), FOR ROUND CROSSBAR
This function returns the bearing stress in the crossbar for a solid round

force is the force in the crossbar
diameter is the diameter of the hole in the crossbar
diameter_crossbar is the diameter of the crossbar
"""
function cross_bar_bearing_stress_round(force, diameter, diameter_crossbar)
    return (force) / (diameter * diameter_crossbar)
end

"""
pin_bearing_stress(force, diameter, thickness), FOR SQUARE CROSSBAR
This function returns the bearing stress in the pin for a square shape crossbar

force is the force in the crossbar
diameter is the diameter of the pin
thickness is the thickness of the crossbar
"""
function pin_bearing_stress(force, diameter, thickness)
    return (force) / (2 * diameter * thickness)
end

"""
pin_bearing_stress(force, diameter, diameter_crossbar), FOR ROUND CROSSBAR
This function returns the bearing stress in the pin for a solid round crossbar

force is the force in the crossbar
diameter is the diameter of the pin
diameter_crossbar is the diameter of the crossbar
"""
function pin_bearing_stress_round(force, diameter, diameter_crossbar)
    return (force) / (diameter * diameter_crossbar)
end

"""
diagonal_bar_tearout_stress(force, length, thickness)
This function returns the tearout stress in the diagonal bar

force is the force in the diagonal member
length is the distance from the center of the hole to the edge of the diagonal member
thickness is the thickness of the diagonal member
"""
function diagonal_bar_tearout_stress(force, length, thickness)
    return (force) / (4 * length * thickness)
end

"""
diagonal_bar_tensile_stress(force, width, diameter, thickness)
This function returns the tensile stress in the diagonal bar

force is the force in the diagonal member
width is the width of the diagonal member
diameter is the diameter of the hole in the diagonal member
thickness is the thickness of the diagonal member
"""
function diagonal_bar_tensile_stress(force, width, diameter, thickness)
    return (force) / ((width - diameter) * thickness * 2)
end

"""
pin_direct_shear(force, n, diameter)
This function returns the direct shear force in the pin

force is the force in the cross bar
n is the number of shear planes in the pin
diameter is the diameter of the pin
"""
function pin_direct_shear(force, n, diameter)
    return (force) / (n * (pi / 4) * (diameter ^ 2))
end

"""
von_mises_stress(sigma_x, sigma_y, tau_xy)
This function returns the von-mises stress from plane stress

sigma_x is the stress in the x direction
sigma_y is the stress in the y direction
tau_xy is the shear in the xy direction
"""
function von_mises_stress(sigma_x, sigma_y, tau_xy)
    return sqrt((sigma_x^2) + (sigma_y^2) - (sigma_x*sigma_y) + (3*(tau_xy^2)))
end

#############################################################################
# Buckling functions:
#############################################################################

function k(I, A)
    return sqrt(I / A)
end

function l_over_k_1(c, E, s_y)
    return sqrt((2*(pi^2)*c*E) / (s_y))
end

function s_r(l, k)
    return l / k
end

"""
max_load_buckling(c, E, s_y, I, A, l)
This function returns the critical loading force for an Euler or Johnson column

c is the end-condition coefficient
E is the modulus of Elasticity
s_y is the yield strength
I is the moment of inertia
A is the area
l is the length
"""
function max_load_buckling(c, E, s_y, I, A, l)
    K = k(I, A)
    lk1 = l_over_k_1(c, E, s_y)
    S_r = s_r(l, K)
    if S_r > lk1
        return (A * c * (pi^2) * E) / (S_r ^ 2)
    else
        return A * (s_y - (1 / (c * E)) * ((s_y * S_r) / (2 * pi))^2)
    end
end
#############################################################################
# Safety Factor functions:
#############################################################################
"""
safety_diagonal_tear(applied_force, S_y_d, l_cb, l_d , l, t_d)
This function calulates the factor of safety for tearout failure in the diagonal members (Currently written for U-bar)

applied_force is the tension force applied to the scissor jack
S_y_d is the yield strength of the diagonal member
l_cb is the length of the cross bar (center of hole to center of hole)
l_d is the length of the diagonal member (center of hole to center of hole)
l is the tearout length of the diagonal member (center of hole to edge)
t_d is the thickness of the diagonal member
"""
function safety_diagonal_tear(applied_force, S_y_d, l_cb, l_d , l, t_d)
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    sigma_nom = diagonal_bar_tearout_stress(F_d, l, t_d)
    sigma_von = von_mises_stress(0.0, 0.0, sigma_nom)
    n = S_y_d / sigma_von
    return n
end

"""
safety_diagonal_axial(applied_force, S_y_d, l_cb, l_d, w_d, d, t_d)
This function calulates the factor of safety for axial failure in the diagonal members (Currently written for U-bar)

applied_force is the tension force applied to the scissor jack
S_y_d is the yield strength of the diagonal member
l_cb is the length of the cross bar (center of hole to center of hole)
l_d is the length of the diagonal member (center of hole to center of hole)
w_d is the width of the diagonal member
d is the diameter of the hole
t_d is the thickness of the diagonal member
"""
function safety_diagonal_axial(applied_force, S_y_d, l_cb, l_d, w_d, d, t_d)
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    sigma_nom = diagonal_bar_tensile_stress(F_d, w_d, d, t_d)
    sigma_von = von_mises_stress(sigma_nom, 0.0, 0.0)
    n = S_y_d / sigma_von
    return n
end

"""
safety_diagonal_bearing(applied_force, S_y_d, l_cb, l_d, d, t_d)
This function calulates the factor of safety for bearing failure in the diagonal members (Currently written for U-bar)

applied_force is the tension force applied to the scissor jack
S_y_d is the yield strength of the diagonal member
l_cb is the length of the cross bar (center of hole to center of hole)
l_d is the length of the diagonal member (center of hole to center of hole)
d is the diameter of the hole
t_d is the thickness of the diagonal member
"""
function safety_diagonal_bearing(applied_force, S_y_d, l_cb, l_d, d, t_d)
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    sigma_nom = diagonal_bar_bearing_stress(F_d, d, t_d)
    sigma_von = von_mises_stress(sigma_nom, 0.0, 0.0)
    n = S_y_d / sigma_von
    return n
end

"""
function safety_crossbar_bearing_round(applied_force, S_y_d, l_cb, l_d, d, d_cb)
This function calulates the factor of safety for bearing failure in the crossbar member (Currently written for solid round crossbar)

applied_force is the tension force applied to the scissor jack
S_y_d is the yield strength of the diagonal member
l_cb is the length of the cross bar (center of hole to center of hole)
l_d is the length of the diagonal member (center of hole to center of hole)
d is the diameter of the hole
d_cb is the diameter of crossbar (assumes crossbar is solid round)
"""
function safety_crossbar_bearing_round(applied_force, S_y_d, l_cb, l_d, d, d_cb)
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    sigma_nom = cross_bar_bearing_stress_round(F_cb, d, d_cb)
    sigma_von = von_mises_stress(sigma_nom, 0.0, 0.0)
    n = S_y_cb / sigma_von
    return n
end

"""
safety_buckling_crossbar(applied_force, S_y_cb, E_cb, l_cb, w_cb, t_cb)
This function calulates the factor of safety for buckling failure in the crossbar (Written for hollow square bar)

applied_force is the tension force applied to the scissor jack
S_y_cb is the yield strength of the crossbar
E_cb is the modulus of elasticity for the crossbar
l_cb is the length of the cross bar (center of hole to center of hole)
l_d is the length of the diagonal member (center of hole to center of hole)
w_cb is the width of the crossbar 
t_cb is the thickness of the crossbar (assumes crossbar is hollow square)
"""
function safety_buckling_crossbar(applied_force, S_y_cb, E_cb, l_cb, l_d, w_cb, t_cb) #for square cross section crossbar
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    A = (w_cb ^ 2) - (w_cb - (2 * t_cb)) ^ 2
    I = ((w_cb ^ 4 / 12)) - ((w_cb - (2 * t_cb)) ^ 4 / 12)
    max_buck = max_load_buckling(C, E_cb, S_y_cb, I, A, l_cb)
    n = max_buck / F_cb
    return n
end

"""
safety_buckling_crossbar_round(applied_force, S_y_cb, E_cb, l_cb, l_d, d_cb)
This function calulates the factor of safety for buckling failure in the crossbar (Written for solid round bar)

applied_force is the tension force applied to the scissor jack
S_y_cb is the yield strength of the crossbar
E_cb is the modulus of elasticity for the crossbar
l_cb is the length of the cross bar (center of hole to center of hole)
l_d is the length of the diagonal member (center of hole to center of hole)
d_cb is the diameter of the crossbar (assumes crossbar is solid round)
"""
function safety_buckling_crossbar_round(applied_force, S_y_cb, E_cb, l_cb, l_d, d_cb) #for solid round cross section crossbar
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    A = ((d_cb / 2) ^ 2) * pi
    I = (pi / 64) * ((d_cb) ^ 4)
    max_buck = max_load_buckling(C, E_cb, S_y_cb, I, A, l_cb)
    n = max_buck / F_cb
    return n
end

"""
safety_pin_shear(applied_force, S_y_p, l_cb, l_d, N, d)
This function calulates the factor of safety for direct shear failure in the pins

applied_force is the tension force applied to the scissor jack
S_y_p is the yield strength of the pin
l_cb is the length of the cross bar (center of hole to center of hole)
l_d is the length of the diagonal member (center of hole to center of hole)
N is the number of shear planes on the pin (should be 2)
d is the diameter of the hole
"""
function safety_pin_shear(applied_force, S_y_p, l_cb, l_d, N, d)
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    tau_nom = pin_direct_shear(F_cb, N, d)
    sigma_von = von_mises_stress(0.0, 0.0, tau_nom)
    n = S_y_p / sigma_von
    return n
end

"""
safety_pin_bearing(applied_force, S_y_p, l_cb, l_d, d, t_cb)
This function calulates the factor of safety for bearing failure in the pin (Currently written for square cross bar)

applied_force is the tension force applied to the scissor jack
S_y_p is the yield strength of the pin
l_cb is the length of the cross bar (center of hole to center of hole)
l_d is the length of the diagonal member (center of hole to center of hole)
d is the diameter of the hole
t_cb is the thickness of the crossbar (assumes crossbar is hollow square)
"""
function safety_pin_bearing(applied_force, S_y_p, l_cb, l_d, d, t_cb)
    F_d, F_cb = scissor_forces(applied_force, l_cb, diagonal_length = l_d)
    sigma_nom = pin_bearing_stress(F_cb, d, t_cb)
    sigma_von = von_mises_stress(sigma_nom, 0.0, 0.0)
    n = S_y_p / sigma_von
    return n
end

safety_pin_bearing

## Pseudocode for search method

for pin in pins

&nbsp; &nbsp; &nbsp; &nbsp; for diag in diagonal_members

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; for cross in crossbars

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; check that pin is compatible with u-bar

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; objective = make_objective(cross, diag, pin, applied_force, end_condition_constant)

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; other optimizer inputs

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; xopt, fopt, info = minimize(objective!, x0, ng, lx, ux, lg, ug, options)

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; get price from outputs

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; if price is < bssf.price

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; update bssf

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; end

&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; end

&nbsp; &nbsp; &nbsp; &nbsp; end

end

display bssf


In [14]:
"""
make_objective(diagonal_member, crossbar, pin, applied_force, end_condition_constant)
This function makes an objective function with the values from the components and loading conditions hard-coded into it.

diagonal_member is a DiagonalMember struct
crossbar is a Crossbar struct
pin is a Pin struct
applied_force is the tension force applied to the jack in lbf
end_condition_constant is the end condition constant for buckling in the crossbar
"""

function make_objective(diagonal_member::DiagonalMember, crossbar::Crossbar, pin::Pin, applied_force, end_condition_constant)
    function objective_function!(x, g)
        #x is a vector that will have:
            #the length of the diagonal member 
            #the length of the crossbar
            #the distance from the center of the hole to the edge of the member in the diagonal member
            #the distance from the center of the hole to the edge of the member in the crossbar
        #here we will "unpack" the vector into variables for legibility
        x[1] = diagonal_length #inches
        x[2] = crossbar_length #inches
        x[3] = diagonal_tearout_length #inches
        x[4] = crossbar_tearout_length #inches
        
        #Calculate safety factors for failure modes (write to items in vector g)
        g[1] #factor of safety for tearout on diagonal member
        g[2] #factor of safety for axial stress on diagonal member
        g[3] #factor of safety for bearing stress on diagonal member
        g[4] #factor of safety for bearing stress on crossbar (is this one still needed?)
        g[5] #factor of safety for tearout on crossbar
        g[6] #factor of safety for buckling on crossbar
        g[7] #factor of safety for shear on pin
        g[8] #factor of safety for bearing stress on pin
        
        #Check geometry (write to item in vector g)
        g[9] #the magnitude of the range of movement the jack has
        
        #Check any other constraints (write to item in vector g) (weight?)

        #Calculate price
        return price
    end
    return objective_function!
end

make_objective (generic function with 1 method)

In [7]:
"""
combo_optimizer(diagonal_member::DiagonalMember, crossbar::Crossbar | CrossbarRound, pin::Pin, safety::SafetyProfile)
This is a function that takes a given combination of a diagonal member, crossbar, pin, and safety profile returns a Solution struct
"""
function combo_optimizer(diagonal_member::DiagonalMember, crossbar::Crossbar, pin::Pin, safety::SafetyProfile)
    #check that the pin is compatable
    if false #replace "false" with a function that checks that the pin is compatible
        return nothing
    end
        
    # make the objective function
    objective! = make_objective(diagonal_member, crossbar, pin, 1000, 1) #double check the applied force

    #set the optimizer variables
    x0 = [1.0, 1.0, 1.0, 1.0]  # starting point
    lx = [0.01, 0.01, 0.01, 0.01]  # lower bounds on x
    ux = [24.0, 48.0, 6.0, 6.0]  # upper bounds on x
    ng = 9  # number of constraints
    lg = [safety.diag_tearout, safety.diag_axial, safety.diag_bearing, safety.cross_bearing, 
            safety.cross_tearout, safety.cross_buckling, safety.pin_shear, safety.pin_bearing, 6]  # lower bounds on g
    ug = Inf*ones(ng)  # upper bounds on g
    options = Options(solver=IPOPT())  # choosing IPOPT solver

    #run the optimizer
    xopt, fopt, info = minimize(objective!, x0, ng, lx, ux, lg, ug, options)

    #build the solution

    #return the solution
end

combo_optimizer